In [ ]:
## This notebook loads the preprocessed fMRI HCP data (nifti files) and uses nilearn tools to define an 
## atlas to parcellate the brain and extract the timeseries from the functional data accounting for cofounds. 
## Then it concatenates the time series from all subjects and saves them as a single output:
## all_subjects_fmri_time_series.csv

In [ ]:
# Given that the HCP resting state data is a relatively long run (1200 volumes) loading the whole dataset 
# occupies a big amount of RAM memory. If you are running this in your local machine this may cause it to crash.
# One way to deal with this problem is to segment the data into smaller chunks of data and
# load each chunk separatelly to extract the time series from it and then concatenate them
# An easy way to do that is using the fslroi function from fsl, by embedding it into a for loop. 
# The following lines of bash code show how you could segment the data this way

# First you will need to rename the nifti files for each subject

# example for the first subject:   
#mv rfMRI_REST1_LR.nii.gz 001.nii.gz

# Then use fslroi in a for loop to segment the data of each subject into 6 chunks, 200 volumes each
##!/bin/sh
# for filename in *.nii.gz ; do
#   fname=`$FSLDIR/bin/remove_ext ${filename}`
#   fslroi ${filename} ${filename}_chunk_01 0 200
#   fslroi ${filename} ${filename}_chunk_02 200 200
#   fslroi ${filename} ${filename}_chunk_03 400 200
#   fslroi ${filename} ${filename}_chunk_04 600 200
#   fslroi ${filename} ${filename}_chunk_05 800 200
#   fslroi ${filename} ${filename}_chunk_06 1000 200
# done

# Once you have prepared the data, you can proceed and run the next cells of this Jupyter notebook 

In [ ]:
# First we set up the dependencies we will be using for running this code
import numpy as np
from nilearn import datasets 
from nilearn import plotting
from numpy import loadtxt
from nilearn.input_data import NiftiLabelsMasker

In [ ]:
# Then we define the atlas to extract timeseries (here Destrieux altas 2009)
dataset = datasets.fetch_atlas_destrieux_2009(lateralized=True) # Select specific altas from downloaded folder
atlas_filename = dataset.maps # Define atlas image
labels = dataset.labels # Set atlas labels that wull be used for plotting
print('Atlas ROIs are located at: %s' % atlas_filename) # Print path to atlas

# Plot atlas with title
plotting.plot_roi(atlas_filename, title="Destrieux atlas")
plotting.show()

In [ ]:
# We create variables containing the elements for the paths to access the data
gen_path = "/media/jonathan/Data/McGill/HCP/" # general path in my pc for the HCP data 
subject_id = ["100307", "102816", "105923", "106521", "108323", "109123", "111514", "112920", "113922", "116524", "116726", "133019"] # HCP subject id
subject = ["001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012"] # subject id
chunks = ["chunk_01", "chunk_02", "chunk_03", "chunk_04", "chunk_05", "chunk_06"] # fMRI data (1200 vols) divided into 6 chunks of 200 vols each
allsub_time_series = np.zeros((1200,148)) # Pre-allocate matrix

# Start for loop to load the confounds txt file from each subject
for i in range(12):
    conf_path = gen_path+subject_id[i]+"/Movement_Regressors.txt"
    conf = loadtxt(conf_path)
    print(conf)
    print(conf.shape)

    # Separate the confound files into chunks that match the fMRI data
    mov_reg1 = conf[range(0,200),:]
    mov_reg2 = conf[range(200,400),:]
    mov_reg3 = conf[range(400,600),:]
    mov_reg4 = conf[range(600,800),:]
    mov_reg5 = conf[range(800,1000),:]
    mov_reg6 = conf[range(1000,1200),:]


    # create paths to load the nifti files from each subject, chunk by chunk
    filenames = []
    for j in range(0,len(chunks)):
        path = gen_path+subject_id[i]+"/"+subject[i]+"_"+chunks[j]+".nii.gz"
        print(path)
        filenames.append(path)
    print(filenames)

    # Extract time series accounting for cofounds, create concatenated matrix
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                               memory='nilearn_cache', verbose=5)

    # Extract the signal time series of each chunk 
    time_series1_c = masker.fit_transform(filenames[0], confounds=mov_reg1)
    time_series2_c = masker.fit_transform(filenames[1], confounds=mov_reg2)
    time_series3_c = masker.fit_transform(filenames[2], confounds=mov_reg3)
    time_series4_c = masker.fit_transform(filenames[3], confounds=mov_reg4)
    time_series5_c = masker.fit_transform(filenames[4], confounds=mov_reg5)
    time_series6_c = masker.fit_transform(filenames[5], confounds=mov_reg6)

    # Concatenate the signal timeseries  
    time_series_c = np.concatenate((time_series1_c,time_series2_c,time_series3_c,time_series4_c,time_series5_c,time_series6_c))
    print(type(time_series_c))
    print(time_series_c.shape)

    # Add this subject timeseries (148x1200 matrix) to a single array containing the timeseries from all subjects
    allsub_time_series = np.concatenate((allsub_time_series,time_series_c))
    